# 0.0 Task

Zaimplementuj algorytm genetyczny z uwzględnieniem poniższych wymagań: 

* kodowanie binarne bądź dyskretne/ciągłe genotypu   
* parametryzacja długości genotypu, wielkości populacji  
* krzyżowanie k-punktowe z prawdopodobieństwem pk, 
* losowanie locusów bez zwracania mutacja m-punktowa z prawdopodobieństwem pm, 
* losowanie locusów bez zwracania selekcja: 
* rankingowa: n najlepszych osobników zastępuje w populacji n najgorszych 
* koło ruletki z uwzględnieniem funkcji transformującej wartości funkcji przystosowania
* warunek stopu: liczba pokoleń, opcjonalnie docelowa wartość funkcji przystosowania, % najlepszych osobników w populacji 
Użyj algorytmu do optymalizacji ANFIS.

# 0.1 Theory about genetic algorithm

It is a search method inspired directly by Charles Darwin's theory of evolution: Survival of the Fittest. 
1. Initialization - creating the first generation, starts by creating a random population of solutions (**chromosomes**)    
2. Then repeat for each generation:  
* Evaluate fitness of each individual  
* Select the best individuals as parents  
* Crossover parent to create children  
* Mutate children with a small chance  
* Termination condition met?  
3. When the condtion for stop was filled, give a solution

# 0.2 Discrete vs. Continious

This is fundamental concept of GA, the representation of the gene dictates how the crossover and mutation operators must work.  

**Discrete Gene** represents a parameter that can only take on a value from a **finite, specific set of option**. The values are distinct and separate, not on a continious spectrum. What it represensts: categorical choices, integer counts or selections from a list

With discrete genes works crossover: **One-point or Two-point Crossover** I can randomly select a gene and replace its value with another valid option from its set.  

**Continious gene** represents a parameter that can take on any real-valued number within a given range. Simple swapping doesn't work as well. Instead, **blending or arithemtic crossover** methods are used. In mutation you can't just swap it for another value. Instead we can add a small amount of random noise. We can use Gaussian Mutation. 

# 0.3 Blend Crossover

What is a **Blend Crossover**? Creates a child gene that is a random value within a range defined by the two parent genes. Example:  
Parent 1 Gene: 0.01  
Parent 2 Gene: 0.05  
Child Gene will be a random number between 0.01 and 0.05  

# 0.4 Arithmetic crossover

What is a **Arithmetic Crossover**? Creates a child gene that is a weigted average of the parents genes.  

$Gene_{child}  = \alpha * Gene_{parent1} + (1-\alpha) * Gene_{parent2}$  
where $\alpha$ is a random weight.

# 0.5 Gaussian mutation

In Gaussian mutation we add a random number from a Gaussian (normal) distribution with a mean of 0 and a small standard deviation.
Example:  
Before mutation: 0.045
After mutation: 0.045 + random_gaussian_noise() = 0.042

## 1.0 Adapting the Genetic Algorithm for ANFIS 

Population of potential schedules. Some are terrible, with a lot of wastted time. Some are pretty good. The schedules with the shortest completion time are more likely to be selected as 'parents'. Between parents like this we would like to make a crossover. Random mutations to ensure we don't get stuck.  
**Gene**: A single operation that needs to be scheduled. Must be uniquely identify, for example (task_id, operation_index_within_task).  
**Chromosome**: A potential solution. A chromosome will be a single, flat list containing all the genes (operations) in a specific order.  

Example:
task 0 : [(T0, R0), (T0, R1)]
task 1: [(T1, R0), (T1, R1)]

Chromosome would be a permutation: [(T0,R0), (T1,R0), (T1, R1), (T0,R1)]  

Fitness function will take a chromosome as an input and calculate fitness score. our goal is to minimize. 

In [1]:
import pandas as pd
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import math
import random
from dataclasses import dataclass
from typing import List, Tuple


@dataclass(frozen=True) # Frozen=True makes instances of this class immutable
class Operation:
    task_id: int #  
    op_id: int # Every task has 11 operations 
    resource: int
    duration: int


class JobShopData:
    """A dedicated class to hold and provide access to data"""
    def __init__(self, path: str):
        df = pd.read_excel('GA_task.xlsx', header=[0, 1])
        
        # A list of all unique operations (genes)
        self.operations: List[Operation] = []
        
        # A structured list of tasks for easy lookup
        self.tasks: List[List[Operation]] = []
        
        # Loop trough task
        for task_id, task_df in df.groupby(level=0, axis=1): # lvl 0 - ID, level 1 - R/T, axis 0 - rows, axis 1 - columns
            # for each task I create a list of its operations
            current_task_operation = []

            r_col = task_df.xs("R", level=1, axis=1).iloc[:,0] # xs stands for cross section, level 0 is the task ID, level 1 is either 'R' or 'T', axis=1 columns
            t_col = task_df.xs("T", level=1, axis=1).iloc[:,0]

            # Loop trough the operations for this task
            op_id_counter = 0
            for resource, time in zip(r_col, t_col):
                # Create operation object
                op = Operation(
                    task_id=task_id,
                    op_id=op_id_counter,
                    resource=resource,
                    duration=time
                )

                # Append new object to the global gene pool
                self.operations.append(op)

                # Append it to the list for the current task
                current_task_operation.append(op)

                op_id_counter += 1

            # After processing all operations for this task and its list to self.tasks
            self.tasks.append(current_task_operation)    

        self.num_tasks = len(self.tasks)
        all_resources = {op.resource for op in self.operations}
        self.num_resources = len(all_resources)
        self.num_operational_total = len(self.operations)

        print(f"Data loaded... \n\nNumber of tasks: {self.num_tasks}\nTootal number of operations: {self.num_operational_total}\nNumber of resources: {self.num_resources}\n\n")


In [2]:
dataload = JobShopData(path='GA_task.xlsx',)

Data loaded... 

Number of tasks: 50
Tootal number of operations: 550
Number of resources: 10




C:\Users\pawel\AppData\Local\Temp\ipykernel_44936\417011900.py:31: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  for task_id, task_df in df.groupby(level=0, axis=1): # lvl 0 - ID, level 1 - R/T, axis 0 - rows, axis 1 - columns


# 2.0 Fitness function

In [3]:
class Scheduler:
    """
    Calculates the fitness of a single chromosome.
    """
    def __init__(self, data: JobShopData):
        """
        Initializes the Scheduler with the problem data. 

        Args:
            data (JobShopData): The loaded job shop problem data.
        """
        self.data = data
    
    def calculate_fitness(self, chromosome: List[Operation]) -> float:
        """
        Calculate the makespan for a given chromosome and returns a fitness score.

        Args:
            chromosome (List[Operation]): A permutation of all operations, representing a schedule's priority list

        Returns:
            float: The fitness score, calculated as 1.0 makespan.
        """
        # The time when resource will be available
        resource_end_times = np.zeros(self.data.num_resources + 1) # Resources are indexed from 1 
        
        # Times when task are over 
        task_end_times = np.zeros(self.data.num_tasks + 1)


        for op in chromosome:
            # Time the task is ready, time of the end of previous operation
            task_ready_time = task_end_times[op.task_id] # In first iteration value is zero

            # Time when resource is available
            resource_ready_time = resource_end_times[op.resource] # In first iteration value is zero

            # An operation can only start when task and resource is ready
            start_time = max(task_ready_time, resource_ready_time)
            end_time = start_time + op.duration

            # Update
            task_end_times[op.task_id] = end_time
            resource_end_times[op.resource] = end_time
        
        # The longest task took:
        total_makespan = np.max(task_end_times) 

        return 1.0 / total_makespan   

In [4]:
scheduler = Scheduler(dataload)

basic_chromosome = dataload.operations

fitness = scheduler.calculate_fitness(basic_chromosome)
makespan = 1 / fitness

print("Scheduler test \n")
print(f"Fitness of basic is: {fitness} and makespan: {makespan}")

Scheduler test 

Fitness of basic is: 8.692628650904033e-05 and makespan: 11504.0


# 3.0 Genetic Algorithm

In [5]:
class GeneticAlgorithm:
    """
    A genetic algorithm implementation.
    """
    def __init__(self,
                data: JobShopData,
                scheduler: Scheduler,
                population_size: int = 350,
                crossover_rate: float = 0.8,
                mutation_rate: float = 0.15,
                selection_method: str = "rank"):
        """
        Initializes instances of class GeneticAlgorithm
        
        Args: 
            data (JobShopData): The job shop data containing operations to be scheduled.
            scheduler (Scheduler): Component that calculates the fitness of chromosomes.
            population_size (int): Number of individuals (chromosomes) are in the population
            crossover_rate (float): The probability that two selected parents will actually breed.
            mutation_rate (float): The probability that an individual gene (operation) in a chromosome will be mutated. 
            selection_method (str): default="roulette" The method used to select parent from the population
        """
        self.data = data
        self.scheduler = scheduler
        self.population_size = population_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population: List[List[Operation]] = []
        self.best_chromosome = None
        self.best_fitness = 0
        
        if selection_method == "roulette":
            self.selection_func = self._selection_roulette
        elif selection_method =="rank": 
            self.selection_func = self._selection_rank
        else:
            raise ValueError("Unknown selection method")

    
    def _create_initial_population(self):
        # Empty list for new population
        new_population = []

        # For each individual
        for population in range(self.population_size):
            # Create a new chromosome
            chromosome = self.data.operations.copy()

            # Schuffle this copy to create a random permuattion
            random.shuffle(chromosome)

            # Add shuffled chromosome to new population
            new_population.append(chromosome)
        
        # Exchanging population
        self.population = new_population

    
    def _selection_roulette(self, fitness_scores: List[float]) -> List[Operation]:
        """
        Selects one parent using Roulette Wheel Selection
        
        Notes:
        This method will have problems when the fitnesses differs very much. 
        """
        # Sum of all fitness scores
        total_fitness = sum(fitness_scores)

        # Picking random number
        pick = random.uniform(0, total_fitness)

        current_sum = 0
        for i, fitness in enumerate(fitness_scores):
            current_sum += fitness
            if current_sum > pick:
                return self.population[i]
        
        
        return self.population[-1]
    

    def _selection_rank(self, fitness_scores: List[float]) -> List[Operation]:
        """Selects one parrent using Rank Selection"""
        ranks = np.argsort(fitness_scores)
        total_rank = self.population_size * (self.population_size + 1) / 2 # mathematical shortcut SUM = N * (N + 1) / 2, formula of sum of the first N integers
        # total rank is like adding weights 1, 2, 3, 4, 5, 6 
        
        pick = random.uniform(0, total_rank)
        
        current_sum = 0
        # ranks tells index of the individual at each rank, so rank[0] is the worst, rank[n-1] is best
        for i, individual_index in enumerate(ranks):
            current_sum += (i + 1)
            if current_sum > pick:
                return self.population[individual_index]
            
        return self.population[ranks[-1]]
    

    def _crossover(self, parent1: List[Operation], parent2: List[Operation]) -> Tuple[List[Operation], List[Operation]]:
        """
        Performs Order Crossover, creates two children
        Notes: 
            Pay attention that in this case we can't use single-point crossover - cut two parent in half and swap the tails because some operation will be missing
            So we have to use crossover method for permutations - it is called order crossover
        """
        size = len(parent1)

        def create_child(p1, p2):
            child = [None] * size
            point1, point2 = random.sample(range(size), 2) # we cant use uniform because it returns float 
            
            # we have to sort this point 
            start = min(point1, point2)
            end = max(point1, point2)

            # Copy the slice from parent1 directly into child1
            slice_from_parent1 = p1[start:end]
            child[start:end] = slice_from_parent1

            # Use a set for checking which geners are already in the child
            genes_in_slice = set(slice_from_parent1)

            # Fill the remaining slots from the second parent (p2)
            p2_idx = end # Starting from the end of the slice
            child_idx = end
            
            # Loop unitl the child is full
            while None in child:
                # If the gene from parent2 is not in the child's slice
                if p2[p2_idx] not in genes_in_slice:
                    # Add it to the child
                    child[child_idx] = p2[p2_idx]

                    child_idx = (child_idx + 1) % size # Why this line it's not over loop? because in case when the gen is in the slice, position to fill
                    # shouldn't move
                p2_idx = (p2_idx + 1) % size

            return child

        child1 = create_child(parent1, parent2)
        child2 = create_child(parent2, parent1)

        return child1, child2

    def _mutation(self, chromosome: List[Operation]):
        """Performs Swap Mutation on a chromosome."""
        if random.random() < self.mutation_rate:
            # Select two indeces for swap
            idx1, idx2 = random.sample(range(len(chromosome)), 2)

            # Swap operations
            chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]


    def run(self, num_generations: int):
        # Create new popultion
        self._create_initial_population()

        for gen in range(num_generations):
            # Evolution
            fitness_scores = [self.scheduler.calculate_fitness(chromo) for chromo in self.population]
            
            # Track best solution
            index_of_best_in_gen = np.argmax(fitness_scores) # argmax returns index of max value
            if fitness_scores[index_of_best_in_gen] > self.best_fitness:
                self.best_fitness = fitness_scores[index_of_best_in_gen]
                self.best_chromosome = self.population[index_of_best_in_gen].copy()

            # Create nexxt generation
            next_generation = []

            # Elitism
            best_in_gen_chromosome = self.population[index_of_best_in_gen]
            next_generation.append(best_in_gen_chromosome.copy())


            while len(next_generation) < self.population_size:
                # Selection
                parent1 = self.selection_func(fitness_scores)
                parent2 = self.selection_func(fitness_scores)

                # Crossover
                if random.random() < self.crossover_rate:
                    new_parent1, new_parent2 = self._crossover(parent1, parent2)
                else:
                    new_parent1, new_parent2 = parent1.copy(), parent2.copy()

                # Mutation
                self._mutation(new_parent1)
                self._mutation(new_parent2)

                offspring1 = new_parent1.copy()
                offspring2 = new_parent2.copy()

                next_generation.append(offspring1)

                # Making sure not to cross tthe limit of population size
                if len(next_generation) < self.population_size:
                    next_generation.append(offspring2)

            self.population = next_generation

            if (gen + 1) % 10 == 0:
                print(f"Generation {gen + 1}/{num_generations}. Best fitness: {self.best_fitness}")

        return self.best_chromosome, self.best_fitness

In [6]:
gen_alg = GeneticAlgorithm(dataload, scheduler)
best_chromosome, best_fitness = gen_alg.run(num_generations=5000)
makespan = 1 / best_fitness
print(f"Best fitness score is equal:{best_fitness} and makespan: {makespan}")

Generation 10/5000. Best fitness: 0.000449842555105713
Generation 20/5000. Best fitness: 0.00045330915684496827
Generation 30/5000. Best fitness: 0.00046382189239332097
Generation 40/5000. Best fitness: 0.00046992481203007516
Generation 50/5000. Best fitness: 0.00047192071731949034
Generation 60/5000. Best fitness: 0.00047192071731949034
Generation 70/5000. Best fitness: 0.00047192071731949034
Generation 80/5000. Best fitness: 0.00047192071731949034
Generation 90/5000. Best fitness: 0.00047192071731949034
Generation 100/5000. Best fitness: 0.00047505938242280285
Generation 110/5000. Best fitness: 0.00047505938242280285
Generation 120/5000. Best fitness: 0.00047505938242280285
Generation 130/5000. Best fitness: 0.00047596382674916705
Generation 140/5000. Best fitness: 0.00047596382674916705
Generation 150/5000. Best fitness: 0.0004768717215069146
Generation 160/5000. Best fitness: 0.0004768717215069146
Generation 170/5000. Best fitness: 0.0004768717215069146
Generation 180/5000. Best fi

In [7]:
best_chromosome

[Operation(task_id=9, op_id=10, resource=2, duration=25),
 Operation(task_id=29, op_id=8, resource=8, duration=33),
 Operation(task_id=46, op_id=5, resource=10, duration=29),
 Operation(task_id=38, op_id=5, resource=1, duration=18),
 Operation(task_id=16, op_id=6, resource=9, duration=21),
 Operation(task_id=5, op_id=5, resource=1, duration=29),
 Operation(task_id=17, op_id=7, resource=4, duration=14),
 Operation(task_id=2, op_id=9, resource=9, duration=12),
 Operation(task_id=8, op_id=5, resource=8, duration=48),
 Operation(task_id=25, op_id=2, resource=7, duration=19),
 Operation(task_id=38, op_id=8, resource=10, duration=24),
 Operation(task_id=38, op_id=6, resource=1, duration=22),
 Operation(task_id=18, op_id=8, resource=5, duration=45),
 Operation(task_id=22, op_id=3, resource=7, duration=33),
 Operation(task_id=5, op_id=7, resource=9, duration=35),
 Operation(task_id=12, op_id=5, resource=8, duration=40),
 Operation(task_id=36, op_id=2, resource=4, duration=24),
 Operation(task_